In [12]:
import requests
#player_id = 35 # just for testing comment it because it will be passed from data factory

#Get player info from this API. construct url
url = f"https://cricket.sportmonks.com/api/v2.0/players/{player_id}?include=career"


headers = {'Content-Type': 'application/json; charset=UTF-8'}
requestBody = {
  'api_token': '7Qk0BLVMSdVLkWP5kjxvjF4OkyFt9urIJAIOxpGOWhrrzwPwTxFcEI5w7zGW'}

#response = requests.post(url, headers=headers, json=requestBody)

#request the url
response = requests.get(url, json=requestBody)
if response.status_code == 200:
  responseBody = response.json()
  #store the response
  data = responseBody['data']
  #print(data)
 
else:
  responseBody = response.json()
  print(responseBody)
  #errors = responseBody['errors']
  #print(errors)


StatementMeta(, c319806a-c017-464a-940d-d1ec124f3b5d, 14, Finished, Available, Finished)

In [13]:
import json
import sys
import pandas as pd
from pyspark.sql.functions import *
from pyspark.sql.types import StringType, DateType, FloatType 



# dumps the json object into an element
json_str = json.dumps(data)

# load the json to a string
resp = json.loads(json_str)

# print the resp
#print(resp)

# extract an element in the response

df = pd.DataFrame(data['career'])
df['id']=resp['id']
df['firstname']=resp['firstname']
df['lastname']=resp['lastname']
df['dateofbirth']=resp['dateofbirth']

#just for display
#display(df)

#construct new dataframe from dataframe (df) with required columns
df2=pd.DataFrame(df['id'])
df2['firstname']=df['firstname']
df2['lastname']=df['lastname']
df2['dateofbirth']=df['dateofbirth']
df2['season_id']=df['season_id']
df2['updated_at']=pd.to_datetime(df['updated_at']).dt.date
df2['type']=df['type']

df2['matches']=df['batting'].str['matches'].astype(float)
df2['runs_scored']=df['batting'].str['runs_scored'].astype(float)
df2['overs']=df['bowling'].str['overs'].astype(float)
df2['wickets']=df['bowling'].str['wickets'].astype(float)

#df3= pd.to_datetime("updated_at") 
#display newly constructed dataframe
#df2.printSchema()
#df3.printSchema()
#display(df2)
'''
# dumps the json object into an element
json_str = pd.json_normalize(df2)
display(json_str)
# load the json to a string
#resp = json.loads(json_str)
'''


StatementMeta(, c319806a-c017-464a-940d-d1ec124f3b5d, 15, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 5594f5b7-525a-4523-a5b2-4aebd2545258)

'\n# dumps the json object into an element\njson_str = pd.json_normalize(df2)\ndisplay(json_str)\n# load the json to a string\n#resp = json.loads(json_str)\n'

In [14]:
#Now save as a table

from pyspark.sql import SparkSession
spark.conf.set('spark.sql.caseSensitive', True)
df = spark.createDataFrame(df2)
df.write.format('delta').mode("append").saveAsTable('FACT_CricketerStats')


StatementMeta(, c319806a-c017-464a-940d-d1ec124f3b5d, 16, Finished, Available, Finished)